In [ ]:
from src.utils import *
from src.dbmongo import DbMongo, get_db
from collections import defaultdict, Counter
from loguru import logger

In [ ]:
#!pip install -r requirements.txt

In [ ]:
def jaccard_index(p1, p2):
    n1, n2 = p1["news"], p2["news"]
    intersection = len((set(n1).intersection(n2)))
    union = (len(n1) + len(n2)) - intersection
    # print(intersection, union)
    return intersection / union

In [ ]:
def is_subset(p1, p2):
    n1, n2 = p1["news"], p2["news"]
    intersection = len((set(n1).intersection(n2)))
    return intersection/len(n1), intersection/len(n2)

In [ ]:
config = parse_config("config.json")
db = get_db(config)

In [ ]:
# update len property
for e in db["entities"].find({}):
    e["len"] = len(e["news"])
    db._upsert_one("entities", e)

In [ ]:
ignore_people = set(config["ignore_people"])

In [ ]:
# delete people that should be ignored
should_be_deleted = [p["_id"] for p in db["people"].find({"_id": {"$in": list(ignore_people)}})]
print(should_be_deleted)

In [ ]:
print(db["people"].delete_many({"_id": {"$in": list(ignore_people)}}))

In [ ]:
explored = [p["_id"] for p in db["people"].find({"processed": True})]
print("explored:", explored)

In [ ]:
# total_news = db["news"].count_documents({})
# total_processed = len(explored)
# print(total_news, total_processed, (total_news/total_processed)*0.05)

In [ ]:
potential = []
for e in db["entities"].find({"label": "PER","len": {"$gt": 15}, "_id": {"$regex": ".+-.+", "$not": {"$in": list(ignore_people) + explored}}}).sort("len", -1).limit(200):
    potential.append(e)
    print(e["_id"], len(e["news"]))
print(len(potential))

In [ ]:
db.upsert_people([{"_id": p["_id"], "name": p["text"]} for p in potential])

In [ ]:
# for p1 in potential:
#     if p1["_id"] in explored: continue
#     for p2 in potential:
#         if p1["_id"] == p2["_id"]: continue
#         if p2["_id"] in explored: continue
        
#         s1, s2 = is_subset(p1, p2)
#         ji = jaccard_index(p1, p2)
# #         if (s1 > 0.6 or s2 >0.6):
#         if ji >= 0.5:
#             print(p1["_id"], p2["_id"], ":", s1, s2, "-", ji)

In [ ]:
# def get_one(name): return next(db["entities"].find({"_id": name}))

In [ ]:
# jaccard_index(get_one("carlos-alexandre"), get_one("cavaco-silva"))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x=[]
for i in range(100):
    x.append(db['entities'].count_documents({"len": i}))

In [ ]:
print("count y entities that appear in x news articles")
n, bins, patches = plt.hist(x, list(range(100)), facecolor='blue', alpha=0.5)
plt.show()

## Check if any website is providing little results

In [ ]:
from collections import Counter

In [ ]:
ws = [n["website"] for n in db["news"].find({"valid": {"$exists": False}, "processed": True}, {"website": 1})]

In [ ]:
Counter(ws)